In [7]:
!pip install gym3
import lib.util

In [8]:
# make module

from lib.masked_attention import MaskedAttention
#REQUIRES:
import functools
import torch as th
from torch import nn
import lib.xf as xf
from lib.minecraft_util import store_args
from lib.tree_util import tree_map

mask_attention_module = MaskedAttention(
        input_size=128,
        memory_size=256,
        heads=2,
        timesteps=128,
        mask = "clipped_causal",
        init_scale=1,
        norm="none",
        log_scope="sa",
        use_muP_factor=False)



In [9]:
# put matrix through module

import torch as th

array = th.normal(0,1,(1000,128,16,16))
print(array.mean())
array2 = array[:,0,None,None]
print(array2.shape)
print(array2.mean())


tensor(-0.0003)
torch.Size([1000, 1, 1, 16, 16])
tensor(0.0007)


In [10]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('facebook/opt-350m')

input_string = ["""A Sunday evening family debate has had my mind whirring over at work. Spurred by the online game Heardle (which is excellent btw)

If the whole of the UK was played 3 - 5 seconds of the intro to one song, what song would be correctly guessed the most?

I argued in favour of the The Bee Gees’ stunning opening riff in Stayin’ Alive, for me, it’s just totally unforgettable."""]

input_ids = tokenizer(input_string)

print(input_ids)

{'input_ids': [[2, 250, 395, 1559, 284, 2625, 34, 56, 127, 1508, 8401, 853, 4506, 81, 23, 173, 4, 2064, 27024, 30, 5, 804, 177, 30689, 459, 36, 5488, 16, 4206, 741, 17137, 43, 50118, 50118, 1106, 5, 1086, 9, 5, 987, 21, 702, 155, 111, 195, 2397, 9, 5, 22845, 7, 65, 2214, 6, 99, 2214, 74, 28, 12461, 28286, 5, 144, 116, 50118, 50118, 100, 3811, 11, 5976, 9, 5, 20, 15227, 4177, 293, 17, 27, 5835, 1273, 31457, 11, 9631, 179, 17, 27, 32190, 6, 13, 162, 6, 24, 17, 27, 29, 95, 4940, 26865, 4]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [85]:
input_ids = tokenizer(""")

If""")
print(input_ids)

input_ids = tokenizer("I want to  eat my bicycle")
print(input_ids)

{'input_ids': [2, 43, 50118, 50118, 1106], 'attention_mask': [1, 1, 1, 1, 1]}
{'input_ids': [2, 100, 236, 7, 1437, 3529, 127, 14678], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
